In [1]:
import pyspark
from pyspark.sql import SparkSession
import gzip
import shutil

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

24/03/02 13:41:40 WARN Utils: Your hostname, psergios-linux resolves to a loopback address: 127.0.1.1; using 192.168.1.205 instead (on interface enp37s0)
24/03/02 13:41:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/02 13:41:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhvhv/fhvhv_tripdata_2021-01.csv.gz

In [ ]:
with gzip.open('fhvhv_tripdata_2021-01.csv.gz', 'rb') as entrada:
    with open('fhvhv_tripdata_2021-01.csv', 'wb') as saida:
        shutil.copyfileobj(entrada, saida)

In [ ]:
!wc -l fhvhv_tripdata_2021-01.csv

In [3]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [ ]:
df.head(5)

In [4]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', StringType(), True), StructField('DOLocationID', StringType(), True), StructField('SR_Flag', StringType(), True)])

In [ ]:
!head -n 101 fhvhv_tripdata_2021-01.csv > head.csv

In [ ]:
!head -n 10 head.csv

In [ ]:
!wc -l head.csv

In [5]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [3]:
import pandas as pd
pd.DataFrame.iteritems = pd.DataFrame.items

/tmp/ipykernel_33276/4172481980.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
df_pandas = pd.read_csv('head.csv')

In [5]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [6]:
spark.createDataFrame(df_pandas).schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [7]:
from pyspark.sql import types

In [8]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [9]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [10]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0003|              B02682|2021-01-01 00:33:44|2021-01-01 00:49:07|         230|         166|   null|
|           HV0003|              B02682|2021-01-01 00:55:19|2021-01-01 01:18:21|         152|         167|   null|
|           HV0003|              B02764|2021-01-01 00:23:56|2021-01-01 00:38:05|         233|         142|   null|
|           HV0003|              B02764|2021-01-01 00:42:51|2021-01-01 00:45:50|         142|         143|   null|
|           HV0003|              B02764|2021-01-01 00:48:14|2021-01-01 01:08:42|         143|          78|   null|
|           HV0005|              B02510|2021-01-01 00:06:59|2021-01-01 00:43:01|

In [11]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 33, 44), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 49, 7), PULocationID=230, DOLocationID=166, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02682', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 55, 19), dropoff_datetime=datetime.datetime(2021, 1, 1, 1, 18, 21), PULocationID=152, DOLocationID=167, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 23, 56), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 38, 5), PULocationID=233, DOLocationID=142, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_datetime=datetime.datetime(2021, 1, 1, 0, 42, 51), dropoff_datetime=datetime.datetime(2021, 1, 1, 0, 45, 50), PULocationID=142, DOLocationID=143, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02764', pickup_dat

In [23]:
df.repartition(24)

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [24]:
df.write.parquet('fhvhv/2021/01')

24/03/01 16:11:02 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 95,00% for 8 writers
24/03/01 16:11:02 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 84,44% for 9 writers
24/03/01 16:11:02 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 76,00% for 10 writers
24/03/01 16:11:02 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
24/03/01 16:11:02 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 63,33% for 12 writers
24/03/01 16:11:11 WARN MemoryManager: Total allocation exceeds 95,00% (1 020 054 720 bytes) of heap memory
Scaling row group sizes to 69,09% for 11 writers
24/03/01 16:11:11 WARN MemoryManager: Total allocation exceeds 95,

In [42]:
df = spark.read.parquet('fhvhv/2021/01')

In [43]:
df

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int, SR_Flag: string]

In [44]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [45]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
.filter(df.hvfhs_license_num == 'HV0003') \
.show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-03 20:20:31|2021-01-03 20:24:27|          62|          61|
|2021-01-03 20:58:26|2021-01-03 21:07:27|          61|          17|
|2021-01-03 20:41:18|2021-01-03 20:45:41|         188|          62|
|2021-01-03 20:49:21|2021-01-03 20:55:43|          62|          62|
|2021-01-03 20:15:22|2021-01-03 20:23:15|          69|         119|
|2021-01-03 20:25:51|2021-01-03 20:34:38|         119|         169|
|2021-01-03 20:38:59|2021-01-03 20:47:24|         169|          18|
|2021-01-03 20:57:13|2021-01-03 21:02:32|          18|         136|
|2021-01-03 20:15:28|2021-01-03 20:21:29|          76|          76|
|2021-01-03 20:24:20|2021-01-03 20:31:17|          76|          77|
|2021-01-03 20:37:12|2021-01-03 20:47:17|         141|         224|
|2021-01-03 20:54:06|2021-01-03 21:01:42|       

In [46]:
from pyspark.sql import functions as F

In [ ]:
#F.to_date()

In [47]:
df \
    .withColumn('pickup_datetime', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_datetime', F.to_date(df.dropoff_datetime)) \
    .show()

+-----------------+--------------------+---------------+----------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|pickup_datetime|dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+---------------+----------------+------------+------------+-------+
|           HV0003|              B02875|     2021-01-03|      2021-01-03|          62|          61|   null|
|           HV0003|              B02875|     2021-01-03|      2021-01-03|          61|          17|   null|
|           HV0003|              B02765|     2021-01-03|      2021-01-03|         188|          62|   null|
|           HV0003|              B02765|     2021-01-03|      2021-01-03|          62|          62|   null|
|           HV0005|              B02510|     2021-01-03|      2021-01-03|         225|         188|   null|
|           HV0005|              B02510|     2021-01-03|      2021-01-03|         189|         189|   null|
|           HV0005|         

In [48]:
df.head(5)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02875', pickup_datetime=datetime.datetime(2021, 1, 3, 20, 20, 31), dropoff_datetime=datetime.datetime(2021, 1, 3, 20, 24, 27), PULocationID=62, DOLocationID=61, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02875', pickup_datetime=datetime.datetime(2021, 1, 3, 20, 58, 26), dropoff_datetime=datetime.datetime(2021, 1, 3, 21, 7, 27), PULocationID=61, DOLocationID=17, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02765', pickup_datetime=datetime.datetime(2021, 1, 3, 20, 41, 18), dropoff_datetime=datetime.datetime(2021, 1, 3, 20, 45, 41), PULocationID=188, DOLocationID=62, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02765', pickup_datetime=datetime.datetime(2021, 1, 3, 20, 49, 21), dropoff_datetime=datetime.datetime(2021, 1, 3, 20, 55, 43), PULocationID=62, DOLocationID=62, SR_Flag=None),
 Row(hvfhs_license_num='HV0005', dispatching_base_num='B02510', pickup_d

In [49]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [51]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
.filter(df.hvfhs_license_num == 'HV0003') \
.show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-03 20:20:31|2021-01-03 20:24:27|          62|          61|
|2021-01-03 20:58:26|2021-01-03 21:07:27|          61|          17|
|2021-01-03 20:41:18|2021-01-03 20:45:41|         188|          62|
|2021-01-03 20:49:21|2021-01-03 20:55:43|          62|          62|
|2021-01-03 20:15:22|2021-01-03 20:23:15|          69|         119|
|2021-01-03 20:25:51|2021-01-03 20:34:38|         119|         169|
|2021-01-03 20:38:59|2021-01-03 20:47:24|         169|          18|
|2021-01-03 20:57:13|2021-01-03 21:02:32|          18|         136|
|2021-01-03 20:15:28|2021-01-03 20:21:29|          76|          76|
|2021-01-03 20:24:20|2021-01-03 20:31:17|          76|          77|
|2021-01-03 20:37:12|2021-01-03 20:47:17|         141|         224|
|2021-01-03 20:54:06|2021-01-03 21:01:42|       